In [2]:
from bio import Animal, AnimalDraw
from ipywidgets import interact


def animal(seed, length, gradation, straightness, distribution, leg_count, ground):
    global ANIMAL
    ANIMAL = Animal.from_params(length, gradation / 10, straightness / 100, distribution / 10, min(leg_count, length - 1), seed)
    return AnimalDraw(ANIMAL).draw(150, ground=ground)


interact(animal, seed=(1, 999999), length=(7, 15),
         gradation=(2.0, 500.), straightness=(1.0, 500.0), distribution=(50.0, 500.0), leg_count=(1, 4),
         ground=(0, 300))


interactive(children=(IntSlider(value=500000, description='seed', max=999999, min=1), IntSlider(value=11, desc…

<function __main__.animal(seed, length, gradation, straightness, distribution, leg_count, ground)>

In [1]:
AnimalDraw(ANIMAL).save_3d('test.stl')

ModuleNotFoundError: No module named 'myplot'